In [ ]:
from scipy.optimize import linprog
import numpy as np

# Debt data: (Loan amount, Interest rate, Min payment)
debts = np.array([
    [5000, 0.15, 200],  # Loan 1: M5,000 at 15% interest, min payment M200
    [3000, 0.12, 150],  # Loan 2: M3,000 at 12% interest, min payment M150
])

# Monthly income & expenses
income = 5000  
expenses = 2500  

# Define LP problem
num_debts = len(debts)
c = debts[:, 1]  # Interest rates (minimize total interest)
A = np.vstack([-np.eye(num_debts), np.ones(num_debts)])  # Constraints for debt payment
b = np.hstack([-debts[:, 2], income - expenses])  # Payments should cover min requirements

# Solve the LP problem
result = linprog(c, A_ub=A, b_ub=b, method="highs")
optimal_payments = result.x  

print("Optimized Debt Payments:", optimal_payments)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np

# Sample Data
data = pd.DataFrame({
    "income": [5000, 6000, 4000, 3000, 7000],
    "expenses": [2500, 3000, 2200, 2700, 3500],
    "debt": [5000, 2000, 7000, 1000, 4000],
    "savings": [500, 1000, 300, 200, 1200],
    "label": ["Stable", "Stable", "At Risk", "Critical", "Stable"]
})

# Split features & labels
X = data.drop(columns=["label"])
y = data["label"]

# Train Decision Tree
model = DecisionTreeClassifier()
model.fit(X, y)

# Predict Financial Status
new_user = np.array([[3000, 3200, 2500, 800]])  # Sample user data
prediction = model.predict(new_user)
print("Financial Status:", prediction)


In [ ]:
import numpy as np
from scipy.optimize import linprog

# Debt Data: (Loan amount, Interest rate (monthly), Min payment, Max payment)
debts = np.array([
    [5000, 0.15/12, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.12/12, 150, 700],   # Loan 2: M3,000 at 12% annual interest
])

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses = 2500  # Fixed monthly expenses
months = 12  # Planning period
num_debts = len(debts)

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((months, num_debts))
remaining_balances = debts[:, 0].copy()

def prioritize_debts(strategy="avalanche"):
    """Returns the sorted index of debts based on the chosen strategy."""
    if strategy == "avalanche":  # Highest interest rate first
        return np.argsort(-debts[:, 1])
    elif strategy == "snowball":  # Lowest balance first
        return np.argsort(debts[:, 0])
    return np.arange(num_debts)

priority_order = prioritize_debts(strategy="avalanche")  # Change to "snowball" if needed

for month in range(months):
    available_cash = income_variability[month] - expenses  # Calculate available cash
    c = debts[:, 1]  # Minimize total interest
    A = np.vstack([-np.eye(num_debts), np.ones(num_debts)])  # Payment constraints
    b = np.hstack([-debts[:, 2], available_cash])  # Min payment & budget limit
    
    # Solve LP for optimal payments
    result = linprog(c, A_ub=A, b_ub=b, bounds=[(debts[i, 2], debts[i, 3]) for i in range(num_debts)], method="highs")
    payments[month, :] = result.x if result.success else debts[:, 2]  # Use min payments if LP fails
    
    # Update remaining balances & compound interest for unpaid balances
    for i in priority_order:
        remaining_balances[i] = max(0, remaining_balances[i] - payments[month, i])
        remaining_balances[i] += remaining_balances[i] * debts[i, 1]  # Apply interest

# Print final optimized payments
print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balances After", months, "Months:")
print(remaining_balances)


In [ ]:
import numpy as np
from scipy.optimize import linprog

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses = 2500  # Fixed monthly expenses
months = 12  # Planning period

# Loan options: (Max Loan Amount, Interest Rate (Annual), Min Payment, Max Payment)
loan_options = np.array([
    [5000, 0.15, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.12, 150, 700],   # Loan 2: M3,000 at 12% annual interest
    [7000, 0.18, 300, 1200],  # Loan 3: M7,000 at 18% annual interest
    [4000, 0.10, 180, 800],   # Loan 4: M4,000 at 10% annual interest
])

# Convert annual interest to monthly interest
loan_options[:, 1] /= 12

# Determine the best loan based on available cash flow
available_cash = np.mean(income_variability) - expenses

def recommend_loan():
    best_loan = None
    best_score = float('inf')
    
    for loan in loan_options:
        max_affordable_payment = min(available_cash, loan[3])
        if max_affordable_payment < loan[2]:
            continue  # Skip unaffordable loans
        
        total_interest = loan[1] * loan[0] * months  # Approximate total interest over period
        score = total_interest / max_affordable_payment  # Lower is better
        
        if score < best_score:
            best_score = score
            best_loan = loan
    
    return best_loan

recommended_loan = recommend_loan()
print("\nRecommended Loan Based on Income & Expenses:")
print(f"Amount: M{recommended_loan[0]:.2f}, Interest Rate: {recommended_loan[1] * 12:.2%}, Min Payment: M{recommended_loan[2]:.2f}, Max Payment: M{recommended_loan[3]:.2f}")

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((months, 1))
remaining_balance = recommended_loan[0]
interest_rate = recommended_loan[1]

for month in range(months):
    payment = min(available_cash, recommended_loan[3])  # Pay as much as possible
    payments[month, 0] = payment
    remaining_balance = max(0, remaining_balance - payment)
    remaining_balance += remaining_balance * interest_rate  # Apply interest

print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balance After", months, "Months:")
print(remaining_balance)

In [ ]:
import numpy as np
from scipy.optimize import linprog

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses_variability = [2500, 2600, 2400, 2300, 2700, 2500, 2550, 2450, 2600, 2550, 2500, 2400]  # Monthly expenses
months = 12  # Planning period

# Loan options: (Max Loan Amount, Interest Rate (Annual), Min Payment, Max Payment)
loan_options = np.array([
    [5000, 0.10, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.30, 150, 700],   # Loan 2: M3,000 at 12% annual interest
    [7000, 0.20, 300, 1200],  # Loan 3: M7,000 at 18% annual interest
    [4000, 0.25, 180, 800],   # Loan 4: M4,000 at 10% annual interest
])

# Convert annual interest to monthly interest
loan_options[:, 1] /= 12

# Determine the best loan based on available cash flow
average_income = np.mean(income_variability)
average_expenses = np.mean(expenses_variability)
available_cash = average_income - average_expenses

def recommend_loan():
    best_loan = None
    best_score = float('inf')
    
    for loan in loan_options:
        max_affordable_payment = min(available_cash, loan[3])
        if max_affordable_payment < loan[2]:
            continue  # Skip unaffordable loans
        
        total_interest = loan[1] * loan[0] * months  # Approximate total interest over period
        score = total_interest / max_affordable_payment  # Lower is better
        
        if score < best_score:
            best_score = score
            best_loan = loan
    
    return best_loan

recommended_loan = recommend_loan()
print("\nRecommended Loan Based on Income & Expenses:")
print(f"Amount: M{recommended_loan[0]:.2f}, Interest Rate: {recommended_loan[1] * 12:.2%}, Min Payment: M{recommended_loan[2]:.2f}, Max Payment: M{recommended_loan[3]:.2f}")

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((months, 1))
remaining_balance = recommended_loan[0]
interest_rate = recommended_loan[1]

for month in range(months):
    available_cash = income_variability[month] - expenses_variability[month]  # Adjust per month
    payment = min(available_cash, recommended_loan[3])  # Pay as much as possible
    payments[month, 0] = payment
    remaining_balance = max(0, remaining_balance - payment)
    remaining_balance += remaining_balance * interest_rate  # Apply interest

print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balance After", months, "Months:")
print(remaining_balance)


In [ ]:
import numpy as np

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses_variability = [2500, 2600, 2400, 2300, 2700, 2500, 2550, 2450, 2600, 2550, 2500, 2400]  # Monthly expenses
months = 12  # Planning period

# Loan options: (Max Loan Amount, Interest Rate (Annual), Min Payment, Max Payment)
loan_options = np.array([
    [5000, 0.15, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.12, 150, 700],   # Loan 2: M3,000 at 12% annual interest
    [7000, 0.18, 300, 1200],  # Loan 3: M7,000 at 18% annual interest
    [4000, 0.10, 180, 800],   # Loan 4: M4,000 at 10% annual interest
])

# Convert annual interest to monthly interest
loan_options[:, 1] /= 12

# Determine the best loan based on available cash flow
average_income = np.mean(income_variability)
average_expenses = np.mean(expenses_variability)
available_cash = average_income - average_expenses

def recommend_loan():
    best_loan = None
    best_amount = 0
    best_months = 0
    best_score = float('inf')
    
    for loan in loan_options:
        max_affordable_payment = min(available_cash, loan[3])
        if max_affordable_payment < loan[2]:
            continue  # Skip unaffordable loans
        
        for term in range(6, 61, 6):  # Loan terms from 6 to 60 months
            monthly_interest = loan[1]
            loan_amount = min(loan[0], max_affordable_payment * term)  # Determine max loan amount
            
            total_interest = 0
            balance = loan_amount
            for _ in range(term):
                interest = balance * monthly_interest
                total_interest += interest
                balance -= max_affordable_payment
                balance += interest  # Add interest to balance
                if balance <= 0:
                    break
            
            score = total_interest / term  # Lower is better
            
            if score < best_score:
                best_score = score
                best_loan = loan
                best_amount = loan_amount
                best_months = term
    
    return best_loan, best_amount, best_months

recommended_loan, recommended_amount, recommended_months = recommend_loan()

print("\nRecommended Loan Based on Income & Expenses:")
print(f"Amount: M{recommended_amount:.2f}, Interest Rate: {recommended_loan[1] * 12:.2%}, Min Payment: M{recommended_loan[2]:.2f}, Max Payment: M{recommended_loan[3]:.2f}, Repayment Period: {recommended_months} months")

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((recommended_months, 1))
remaining_balance = recommended_amount
interest_rate = recommended_loan[1]

for month in range(recommended_months):
    available_cash = income_variability[month % len(income_variability)] - expenses_variability[month % len(expenses_variability)]  # Adjust per month
    payment = min(available_cash, recommended_loan[3])  # Pay as much as possible
    payments[month, 0] = payment
    remaining_balance = max(0, remaining_balance - payment)
    remaining_balance += remaining_balance * interest_rate  # Apply interest

print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balance After", recommended_months, "Months:")
print(remaining_balance)

In [ ]:
import numpy as np

# ========== User Inputs ==========
# Cash flow history (monthly figures)
income_history = np.array([5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100])
expense_history = np.array([2500, 2600, 2400, 2300, 2700, 2500, 2550, 2450, 2600, 2550, 2500, 2400])
months_history = len(income_history)

# Target savings (monthly)
target_savings = 800  # e.g., M800 saved per month

# Assets: investments, properties, etc.
# You might have a structured dictionary of asset types and values.
assets = {
    'investments': 15000,
    'property': 50000
}
total_assets = sum(assets.values())

# Collateral factor (e.g., you decide you want the maximum loan to be 50% of total assets)
collateral_factor = 0.5
max_loan_by_assets = total_assets * collateral_factor

# ========== Loan Option Data ==========
# Each row: [Min Loan, Max Loan, Annual Interest, Minimum Monthly Payment, Maximum Monthly Payment]
loan_options = np.array([
    [1000, 5000, 0.15, 200, 1000],
    [800, 3000, 0.12, 150, 700],
    [2000, 7000, 0.18, 300, 1200],
    [500, 4000, 0.10, 180, 800],
], dtype=float)

# Convert annual interest to monthly interest for each option:
loan_options[:, 2] /= 12

# ========== Step 1: Determine Financial Flexibility ==========
# Compute average income and expenses from historical data
avg_income = np.mean(income_history)
avg_expenses = np.mean(expense_history)
avg_discretionary = avg_income - avg_expenses - target_savings

print("Average Income: M{:.2f}".format(avg_income))
print("Average Expenses: M{:.2f}".format(avg_expenses))
print("Target Savings: M{:.2f}".format(target_savings))
print("Available monthly surplus for loan repayment: M{:.2f}".format(avg_discretionary))

# Advise whether a loan is even necessary.
if avg_discretionary <= 0:
    print("\nBased on your cash flow, taking on additional debt may strain your ability to meet your savings targets.")
    advise = "Avoid taking a new loan; consider cutting expenses or increasing your income."
else:
    advise = "A loan might be feasible given your surplus cash flow."
print("\nAdvice on taking a loan:")
print(advise)

# ========== Step 2: Determine Safe Loan Range ==========
# Define safe loan range:
#   - Lower bound: the minimum that would make a difference (could be based on specific needs)
#   - Upper bound: the minimum of the maximum allowed by the loan option and what your assets cover
safe_loan_lower = 0  # Possibly, if you need any extra working capital, determine a minimum required amount
safe_loan_upper = max_loan_by_assets
print("\nBased on your assets, you can safely collateralize up to: M{:.2f}".format(safe_loan_upper))

# ========== Step 3: Loan Option Simulation ==========
# For each loan option, and for a range of terms and loan amounts, simulate the monthly repayment schedule.
def simulate_repayment(principal, monthly_rate, term, monthly_payment):
    """
    Simulate the amortization of a loan given:
      - principal: loan amount
      - monthly_rate: monthly interest rate
      - term: number of months over which repayment is made
      - monthly_payment: fixed monthly payment amount
    Returns total interest paid and final balance (if any)
    """
    balance = principal
    total_interest = 0.0
    for _ in range(term):
        interest = balance * monthly_rate
        total_interest += interest
        balance = balance + interest - monthly_payment
        if balance <= 0:
            break
    return total_interest, balance

# We want to choose the option that minimizes a penalty function such as:
#    penalty = (total_interest/term) + lambda * (shortfall in maintaining target surplus)
# where lambda is a factor that weighs the importance of saving targets.
lambda_penalty = 1.0  # adjust based on risk tolerance and target importance

results = []
term_range = range(6, 61, 6)  # consider terms from 6 months to 60 months

# For each loan option, simulate over a grid of principal amounts and terms.
for idx, option in enumerate(loan_options):
    min_loan, max_loan, monthly_rate, min_payment, max_payment = option
    # Adjust maximum possible loan for this option by both the product constraints and your collateral
    option_max = min(max_loan, safe_loan_upper)
    # Use a small grid for loan amounts: from min_loan to option_max (if option_max is higher than min_loan)
    if option_max < min_loan:
        continue  # Skip options that you cannot use because the upper bound is below the lender's minimum.
    principal_grid = np.linspace(min_loan, option_max, num=5)  # 5 sample points
    for term in term_range:
        for principal in principal_grid:
            # Determine what monthly payment can be "afforded" given your surplus and the option limits.
            # Here, we assume you want to use up to your available surplus but not exceeding the max allowed by the loan.
            feasible_payment = min(avg_discretionary, max_payment)
            # But the payment must at least be the lender's minimum requirement.
            if feasible_payment < min_payment:
                continue  # if the payment isn't high enough to satisfy the minimum payment requirement.
            total_interest, final_balance = simulate_repayment(principal, monthly_rate, term, feasible_payment)
            # Compute an adjustment penalty if the payment structure forces you to dip into your savings target.
            # For simulation purposes, if the feasible payment is below what is required to amortize the loan,
            # you might “penalize” that option.
            if final_balance > 0:
                penalty = 1000  # heavy penalty if the loan would not be paid off in term
            else:
                penalty = 0
            score = (total_interest/term) + lambda_penalty * penalty
            results.append({
                "option_index": idx,
                "loan_parameters": option,
                "term": term,
                "principal": principal,
                "monthly_payment": feasible_payment,
                "avg_interest_per_month": total_interest/term,
                "score": score
            })

# ========== Step 4: Selection & Recommendation ==========
if not results:
    print("\nNo loan option meets the feasibility criteria based on your current financial profile.")
else:
    # Choose the result with the lowest score.
    best_result = min(results, key=lambda x: x["score"])
    print("\n--- Loan Optimization Recommendation ---")
    option_used = loan_options[int(best_result["option_index"])]
    print("Recommended Loan Option:")
    print(" - Loan Details:")
    print("     * Minimum Loan: M{:.2f}, Maximum Loan: M{:.2f}".format(option_used[0], option_used[1]))
    print("     * Annual Interest Rate: {:.2%}".format(option_used[2] * 12))
    print("     * Payment Range: M{:.2f} - M{:.2f} per month".format(option_used[3], option_used[4]))
    print(" - Recommended Principal: M{:.2f}".format(best_result["principal"]))
    print(" - Recommended Term: {} months".format(best_result["term"]))
    print(" - Estimated Average Interest Cost per Month: M{:.2f}".format(best_result["avg_interest_per_month"]))
    print(" - Monthly Payment (affordable): M{:.2f}".format(best_result["monthly_payment"]))
    print("\nNote: The simulation also takes into account your target saving requirement and the asset-based collateral constraints.")


In [ ]:
import pandas as pd
import numpy as np

# Simulate training dataset
np.random.seed(42)

n_samples = 200

# Simulate income and expenses
monthly_income = np.random.randint(3000, 15000, n_samples)
monthly_expenses = np.random.randint(1000, 8000, n_samples)
monthly_surplus = monthly_income - monthly_expenses

# Assets
total_assets_value = np.random.randint(1000, 50000, n_samples)

# Loan options
interest_rate = np.random.uniform(5, 25, n_samples)
min_amount = np.random.randint(1000, 10000, n_samples)
max_amount = min_amount + np.random.randint(1000, 15000, n_samples)
term_months = np.random.choice([6, 12, 18, 24, 36], n_samples)

# Goal
target_savings = np.random.randint(5000, 50000, n_samples)
time_horizon = np.random.choice([6, 12, 18, 24], n_samples)

# Calculate a naive loan amount (midpoint)
loan_amount = (min_amount + max_amount) / 2

# Calculate monthly loan payment using a basic amortized formula
monthly_loan_payment = loan_amount * (interest_rate / 100) / 12

# Calculate achievable savings
achievable_savings = (monthly_surplus - monthly_loan_payment) * time_horizon + loan_amount

# Savings margin (target)
savings_margin = achievable_savings - target_savings

# Binary target: is this loan plan optimal?
is_optimal = (savings_margin >= 0).astype(int)

# Assemble DataFrame
df = pd.DataFrame({
    'average_monthly_income': monthly_income,
    'average_monthly_expenses': monthly_expenses,
    'monthly_surplus': monthly_surplus,
    'total_assets_value': total_assets_value,
    'interest_rate': interest_rate,
    'min_amount': min_amount,
    'max_amount': max_amount,
    'term_months': term_months,
    'target_savings': target_savings,
    'time_horizon': time_horizon,
    'loan_amount': loan_amount,
    'monthly_loan_payment': monthly_loan_payment,
    'achievable_savings': achievable_savings,
    'savings_margin': savings_margin,
    'is_optimal': is_optimal,
})

df.head()


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

model = RandomForestRegressor(n_estimators=100)

X = df[[
    'average_monthly_income', 'average_monthly_expenses', 'total_assets_value',
    'interest_rate', 'min_amount', 'max_amount', 'term_months', 'target_savings', 'time_horizon'
]]

y = df['savings_margin']  # or binary label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model.fit(X_train, y_train)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# ----- Step 1: Generate a simulated dataset -----

np.random.seed(42)
n_samples = 200

# Simulate cashflow details
monthly_income = np.random.randint(3000, 15000, n_samples)
monthly_expenses = np.random.randint(1000, 8000, n_samples)
monthly_surplus = monthly_income - monthly_expenses

# Simulate asset data
total_assets_value = np.random.randint(1000, 50000, n_samples)

# Simulate loan option details
interest_rate = np.random.uniform(5, 25, n_samples)
min_amount = np.random.randint(1000, 10000, n_samples)
max_amount = min_amount + np.random.randint(1000, 15000, n_samples)
term_months = np.random.choice([6, 12, 18, 24, 36], n_samples)

# Simulate savings goals
target_savings = np.random.randint(5000, 50000, n_samples)
time_horizon = np.random.choice([6, 12, 18, 24], n_samples)

# Calculate a naive loan amount (midpoint)
loan_amount = (min_amount + max_amount) / 2

# Calculate monthly loan payment using a simple approximation
monthly_loan_payment = loan_amount * (interest_rate / 100) / 12

# Calculate achievable savings based on surplus, monthly payment and time horizon
achievable_savings = (monthly_surplus - monthly_loan_payment) * time_horizon + loan_amount

# Savings margin (difference between achievable savings and target savings)
savings_margin = achievable_savings - target_savings

# Binary target: 1 if achievable savings meets/exceeds target, else 0
is_optimal = (savings_margin >= 0).astype(int)

# Assemble DataFrame
df = pd.DataFrame({
    'average_monthly_income': monthly_income,
    'average_monthly_expenses': monthly_expenses,
    'monthly_surplus': monthly_surplus,
    'total_assets_value': total_assets_value,
    'interest_rate': interest_rate,
    'min_amount': min_amount,
    'max_amount': max_amount,
    'term_months': term_months,
    'target_savings': target_savings,
    'time_horizon': time_horizon,
    'loan_amount': loan_amount,
    'monthly_loan_payment': monthly_loan_payment,
    'achievable_savings': achievable_savings,
    'savings_margin': savings_margin,
    'is_optimal': is_optimal,
})

print("First 5 rows of the dataset:")
print(df.head())

# ----- Step 2: Train/Test Split and Model Training -----

X = df[[ 
    'average_monthly_income',
    'average_monthly_expenses',
    'total_assets_value',
    'interest_rate',
    'min_amount',
    'max_amount',
    'term_months',
    'target_savings',
    'time_horizon'
]]
y = df['savings_margin']  # Using the continuous savings_margin value

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate and train the RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ----- Step 3: Evaluate the Model on Test Set -----

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Compute evaluation metrics
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("\nEvaluation on Test Set:")
print("R2 Score:", r2)
print("RMSE:", rmse)

# ----- Step 4: Generate New Testing Data and Use the Model -----

# Here is an example of new loan option data (as a DataFrame)
new_test_data = pd.DataFrame({
    'average_monthly_income': [8000, 12000],
    'average_monthly_expenses': [4000, 3500],
    'total_assets_value': [15000, 25000],
    'interest_rate': [10.0, 12.5],
    'min_amount': [5000, 7000],
    'max_amount': [8000, 10000],
    'term_months': [12, 24],
    'target_savings': [20000, 30000],
    'time_horizon': [12, 24]
})

# Generate predictions for the new testing data
new_predictions = model.predict(new_test_data)
print("\nPredictions for new test data (savings_margin):")
print(new_predictions)

# Optionally, if you want to assess optimality based on savings_margin being non-negative:
new_optimal = new_predictions >= 0
print("\nAre the new loan options optimal?")
print(new_optimal)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, 
    mean_squared_error, 
    mean_absolute_error, 
    explained_variance_score,
    accuracy_score,
    confusion_matrix,
    classification_report
)

# Predictions on the test set
y_pred = model.predict(X_test)

# Calculate regression metrics
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)

print("\n--- Regression Metrics ---")
print("R2 Score:", r2)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("Explained Variance Score:", explained_var)

# ----- Step 4: Evaluate the Model as a Classifier for Optimality -----
# By thresholding predictions: consider predictions with savings_margin >= 0 as optimal

threshold = 0
# Convert continuous predictions to binary optimality decisions
y_test_class = (y_test >= threshold).astype(int)
y_pred_class = (y_pred >= threshold).astype(int)

# Compute classification metrics
accuracy = accuracy_score(y_test_class, y_pred_class)
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
class_report = classification_report(y_test_class, y_pred_class)

print("\n--- Classification Metrics for Optimality (savings_margin >= 0) ---")
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

In [ ]:
import pandas as pd

# Define the personal financial profile (remains constant)
user_profile = {
    'total_assets_value': 25000,
    'average_monthly_income': 12000,
    'average_monthly_expenses': 4000,
    'target_savings': 30000
}

# Define multiple loan options in a DataFrame
loan_options = pd.DataFrame({
    'min_amount': [5000, 7000, 6000],
    'interest_rate': [10.0, 12.5, 11.0],
    'max_amount': [8000, 10000, 9000],
    'term_months': [12, 24, 18]
})

# Attach the user's financial profile to each loan option row
loan_options['total_assets_value'] = user_profile['total_assets_value']
loan_options['average_monthly_income'] = user_profile['average_monthly_income']
loan_options['average_monthly_expenses'] = user_profile['average_monthly_expenses']
loan_options['target_savings'] = user_profile['target_savings']

# Display the combined DataFrame for testing or model input
loan_options


In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# ===============================
# TRAINING PHASE: Create Synthetic Dataset and Train Model
# ===============================

np.random.seed(42)
n_samples = 200

# --- Personal Financial Data (varying across training samples) ---
average_monthly_income = np.random.randint(3000, 15000, n_samples)
average_monthly_expenses = np.random.randint(1000, 8000, n_samples)
total_assets_value = np.random.randint(1000, 50000, n_samples)
target_savings = np.random.randint(5000, 50000, n_samples)
time_horizon = np.random.choice([6, 12, 18, 24], n_samples)

# --- Loan Option Details ---
min_amount = np.random.randint(1000, 10000, n_samples)
max_amount = min_amount + np.random.randint(1000, 15000, n_samples)
interest_rate = np.random.uniform(5, 25, n_samples)
term_months = np.random.choice([6, 12, 18, 24, 36], n_samples)
# Loan names
loan_names = np.random.choice(
    ['Starter Loan', 'AgriFlex', 'Business Boost', 'SmartSaver', 'GreenGrowth', 'HomeEase', 'QuickFund', 'EduPlan'],
    n_samples
)

# --- Derived Calculations ---
# Loan amount is set as the midpoint between min and max amounts:
loan_amount = (min_amount + max_amount) / 2

# Estimate monthly loan payment using a simple approximation:
monthly_loan_payment = loan_amount * (interest_rate / 100) / 12

# Calculate monthly surplus:
monthly_surplus = average_monthly_income - average_monthly_expenses

# Calculate achievable savings:
achievable_savings = (monthly_surplus - monthly_loan_payment) * time_horizon + loan_amount

# Savings margin is the difference between achievable savings and target savings:
savings_margin = achievable_savings - target_savings

# The regression target will be the continuous savings margin:
y = savings_margin

# Assemble the training features into a DataFrame
df_train = pd.DataFrame({
    'average_monthly_income': average_monthly_income,
    'average_monthly_expenses': average_monthly_expenses,
    'total_assets_value': total_assets_value,
    'interest_rate': interest_rate,
    'min_amount': min_amount,
    'max_amount': max_amount,
    'term_months': term_months,
    'target_savings': target_savings,
    'time_horizon': time_horizon,
    "loan_names": loan_names,
})

print("Training DataFrame (first 5 rows):")
df_train.head()





Training DataFrame (first 5 rows):


,average_monthly_income,average_monthly_expenses,total_assets_value,interest_rate,min_amount,max_amount,term_months,target_savings,time_horizon,loan_names
0,10270,4445,1699,22.891045,1055,9251,24,29052,18,AgriFlex
1,3860,4743,1190,20.997105,9717,22442,12,43513,18,EduPlan
2,8390,6727,11492,13.504270,9150,21709,6,31092,6,EduPlan
3,8191,2495,36743,5.449386,3125,7493,6,16338,6,AgriFlex
4,14964,4304,7102,10.373547,1364,6161,36,5412,6,SmartSaver


In [ ]:
# ----- Split the Data and Train the Model -----
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model performance on the test set
y_pred = model.predict(X_test)
print("\n--- Model Evaluation on Test Set ---")
print("R2 Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# --- Preprocessing: One-Hot Encoding for loan_names ---
# Convert loan_names to one-hot encoded columns
column_transformer = ColumnTransformer(
    transformers=[
        ('loan_names', OneHotEncoder(), ['loan_names'])  # Apply OneHotEncoder to loan_names column
    ], 
    remainder='passthrough'  # Keep the rest of the columns as they are
)

# --- Define the model pipeline ---
pipeline = Pipeline(steps=[
    ('preprocessor', column_transformer),  # Preprocess the data
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))  # Train using RandomForest
])



y = df_train['target_savings']
X = df_train.drop(columns=['target_savings'])


# --- Train the model ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# --- Evaluate the model ---
y_pred = pipeline.predict(X_test)

# --- Calculate Metrics ---
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Absolute Error: 14094.752000000002
Mean Squared Error: 263681516.01436996
R-squared: -0.24884435895993584


In [23]:
# ===============================
# PREDICTION PHASE: Using the Trained Model
# ===============================

# --- Step 1: Define the Constant User Financial Profile ---
user_profile = {
    'total_assets_value': 25000,
    'average_monthly_income': 12000,
    'average_monthly_expenses': 4000,
    'target_savings': 30000,
    'time_horizon': 12  # Example: 12 months
}

# --- Step 2: Define Multiple Loan Options to Evaluate ---
# Each row represents one loan option candidate.
loan_options = pd.DataFrame({
    'min_amount': [5000, 7000, 6000],
    'interest_rate': [10.0, 12.5, 11.0],
    'max_amount': [8000, 10000, 9000],
    'term_months': [12, 24, 18],
    'loan_names': ['Business Boost', 'SmartSaver', 'AgriFlex']  # Example loan names
})

# --- Step 3: Calculate the Derived Loan Amount and Monthly Payment ---
loan_options['loan_amount'] = (loan_options['min_amount'] + loan_options['max_amount']) / 2
loan_options['monthly_loan_payment'] = loan_options['loan_amount'] * (loan_options['interest_rate'] / 100) / 12

# --- Step 4: Calculate Monthly Surplus ---
monthly_surplus = user_profile['average_monthly_income'] - user_profile['average_monthly_expenses']

# --- Step 5: Calculate Achievable Savings ---
loan_options['achievable_savings'] = (monthly_surplus - loan_options['monthly_loan_payment']) * user_profile['time_horizon'] + loan_options['loan_amount']

# --- Step 6: Calculate Savings Margin ---
loan_options['savings_margin'] = loan_options['achievable_savings'] - user_profile['target_savings']

# --- Step 7: Prepare the Prediction Data ---
# Create a DataFrame of user profile information and loan options
X_predict = pd.DataFrame({
    'total_assets_value': [user_profile['total_assets_value']] * len(loan_options),
    'average_monthly_income': [user_profile['average_monthly_income']] * len(loan_options),
    'average_monthly_expenses': [user_profile['average_monthly_expenses']] * len(loan_options),
    'interest_rate': loan_options['interest_rate'],
    'min_amount': loan_options['min_amount'],
    'max_amount': loan_options['max_amount'],
    'term_months': loan_options['term_months'],
    'target_savings': [user_profile['target_savings']] * len(loan_options),
    'time_horizon': [user_profile['time_horizon']] * len(loan_options),
    'loan_names': loan_options['loan_names']
})

# --- Step 8: Preprocess and Make Predictions ---
# Use the trained pipeline to predict the savings margin
predicted_savings_margin = pipeline.predict(X_predict)

# --- Step 9: Display the Predictions ---
loan_options['predicted_savings_margin'] = predicted_savings_margin
print("Loan Options with Predicted Savings Margin:")
print(loan_options[['loan_names', 'predicted_savings_margin']])


Loan Options with Predicted Savings Margin:
       loan_names  predicted_savings_margin
0  Business Boost                  24899.04
1      SmartSaver                  23501.90
2        AgriFlex                  26316.77


In [ ]:
# ===============================
# PREDICTION PHASE: Evaluate Multiple Loan Options for a Single User Profile
# ===============================

# --- Step 1: Define the Constant User Financial Profile ---
user_profile = {
    'total_assets_value': 25000,
    'average_monthly_income': 12000,
    'average_monthly_expenses': 4000,
    'target_savings': 30000
}

# --- Step 2: Define Multiple Loan Options to Evaluate ---
# Each row represents one loan option candidate.
loan_options = pd.DataFrame({
    'min_amount': [5000, 7000, 6000],
    'interest_rate': [10.0, 12.5, 11.0],
    'max_amount': [8000, 10000, 9000],
    'term_months': [12, 24, 18],
})

# Assume a fixed time horizon for prediction (if required by the model)
loan_options['time_horizon'] = 12

# --- Step 3: Attach the User Profile to Each Loan Option ---
loan_options['total_assets_value'] = user_profile['total_assets_value']
loan_options['average_monthly_income'] = user_profile['average_monthly_income']
loan_options['average_monthly_expenses'] = user_profile['average_monthly_expenses']
loan_options['target_savings'] = user_profile['target_savings']

# Reorder columns to match the training features order:
features_order = [
    'average_monthly_income',
    'average_monthly_expenses',
    'total_assets_value',
    'interest_rate',
    'min_amount',
    'max_amount',
    'term_months',
    'target_savings',
    'time_horizon'
]
loan_options = loan_options[features_order]

print("\nLoan Options for Evaluation:")
print(loan_options)

# --- Step 4: Predict Savings Margin for Each Loan Option ---
predictions = model.predict(loan_options)
loan_options['predicted_savings_margin'] = predictions

print("\nPredicted Savings Margins for Each Loan Option:")
print(loan_options[['min_amount', 'interest_rate', 'max_amount', 'term_months', 'predicted_savings_margin']])

# --- Step 5: Select the Best Loan Option ---
# Here, the best option is defined as the one with the highest predicted savings margin.
best_option_index = loan_options['predicted_savings_margin'].idxmax()
best_loan = loan_options.loc[best_option_index]
print("\nBest Loan Option Based on Predicted Savings Margin:")
print(best_loan)
